In [1]:
from bs4 import BeautifulSoup
import requests
import pickle
from lxml import html
from Swimmer import *
import selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

## Inserire il codice dello stile che si desidera selezionare
### Stile Libero:
###### "00" 50mt, "01" 100mt, "02" 200mt, "04" 400mt, "08" 800mt, "15" 1500mt
### Dorso:
##### "20" 50mt, "21" 100mt, "22" 200mt
### Rana:
##### "30" 50mt, "31" 100mt, "32" 200mt
### Farfalla:
##### "40" 50 mt, "41" 100mt, "42" 200mt
### Misti:
##### "51" 100mt, "52" 200mt, "54" 400mt

In [3]:
Stile = "02"

## Inserire lunghezza vascha

In [4]:
Vasc = "50"

## Inserire sesso dei nuotatori

In [5]:
Sex = "M"

In [6]:
# Creaiamo una risorsa Browser per poter scaricare i dati
url = "http://online.federnuoto.it/graduatorie/mas_atleti.php"

driver = webdriver.Firefox()

driver.get(url)

In [7]:
# Selezioniamo le Form e inseriamo i dati al loro interno
righe = driver.find_element_by_name("txtnurighe")
righe.clear()
righe.send_keys("999")
annoag = Select(driver.find_element_by_name("txtanno"))
annoag.select_by_value("ALLTIME")
annoin = driver.find_element_by_name("txtdallanno")
annoin.clear()
annoin.send_keys("1900")
annofin = driver.find_element_by_name("txtallanno")
annofin.clear()
annofin.send_keys("1967")
sesso = Select(driver.find_element_by_name("txtsesso"))
sesso.select_by_value(Sex)
vasca = Select(driver.find_element_by_name("txtvasca"))
vasca.select_by_value(Vasc)
gara = Select(driver.find_element_by_name("txtgara"))
gara.select_by_value(Stile)

# Simuliamo l'inserimento dei dati e salviamo il risultato ottenuto
driver.find_element_by_name("xricerca").click()

In [8]:
# Salviamo i sorgenti della pagina
content = driver.page_source

with open("nuotatori_"+Stile+".html", "w") as f:
    f.write(content)
    
driver.close()

In [9]:
# Apriamo il file html
with open("nuotatori_"+Stile+".html") as file:
    dirty_strings = file.read()

In [10]:
dirty_strings = dirty_strings.split("onclick=\"window.open('")

# Filtra i link per ogni nuotatore
dirty_strings = list(filter(lambda x: "mas_schedaris.php" == x[:len("mas_schedaris.php")],
                            dirty_strings))


In [11]:
# Crea dei link validi
clean_strings = [x.split("\'")[0] for x in dirty_strings]

final_strings = ["http://online.federnuoto.it/graduatorie/"+"&".join(x.split(";")) for x in clean_strings]

In [12]:
# Salva i link validi
with open("nuotatori_"+Stile+"_filtrati.html","w") as file:
    for link in final_strings:
        file.write(link+"\n")

In [13]:
# Funzione che dato un link ci ritorna il nome del nuotatore
def get_name(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    a = a.split("""color:#000080">""")
    return a[1].split("(")[0].strip()

In [14]:
# Funzione che dato un link ci ritorna l'anno di nascita del nuotatore
def get_class(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    a = a.split("""color:#000080">""")
    return a[1].split("(")[1][:4]

In [15]:
# Funzione che dato un link ci ritorna data e tempo di ogni gara del nuotatore
def get_times_date(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    first_split = a.split("""<td align="center" height="20" style=" font-family:Verdana, Arial, Helvetica, sans-serif; font-weight:normal; font-style:normal; font-size:10px; color:#000080" width="10%">""")
    s = """</td> <td align="left" height="20" style=" font-family:Verdana, Arial, Helvetica, sans-serif; font-weight:normal; font-style:normal; font-size:10px; color:#000080" width="40%">"""
    second_split=[]

    for i in first_split:
        if s in i:
            second_split.append((i[2:10],i[13+len(s):len(s)+23]))
    
    return second_split

In [16]:
# Funzione che dato un link ci ritorna luogo di ogni gara del nuotatore
def get_locality(url):
    r = requests.get(url)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    a = soup.prettify().replace("\n", "")
    first_split = a.split("""</td> <td align="left" height="20" style=" font-family:Verdana, Arial, Helvetica, sans-serif; font-weight:normal; font-style:normal; font-size:10px; color:#000080" width="40%">""")
    s="""<title>   Federazione Italiana Nuoto"""
    split=[]

    for i in first_split:
        c = 0
        while i[15+c]!= "-":
            c = c + 1
        if i[15:15+(c-1)]!=s:
            split.append(i[15:15+(c-1)])
        
    return split

In [17]:
# Creo una lista di nuotatori
swimmers = []
i = 1
with open("nuotatori_"+Stile+"_filtrati.html") as html:
    for j in html:
        url = j.strip()
        s = Swimmer()
        s.url = url
        s.name = get_name(url)
        s.times_date = get_times_date(url)
        s.year = get_class(url)
        s.locality = get_locality(url)
        swimmers.append(s)
        print(str(i) + ".", end="")
        if i % 25 == 0:
            print()
        i += 1
     

1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.
26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.
51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.
76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.
101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.124.125.
126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.
151.152.153.154.155.156.157.158.159.160.161.162.163.164.165.166.167.168.169.170.171.172.173.174.175.
176.177.178.179.180.181.182.183.184.185.186.187.188.189.190.191.192.193.194.195.196.197.198.199.200.
201.202.203.204.205.206.207.208.209.210.211.212.213.214.215.216.217.218.219.220.221.222.223.224.225.
226.227.228.229.230.231.232.233.234.235.236.237.238.239.240.241.242.243.244.245.246.247.248.249.250.
251.252.253.254.255.256.257.258.259.260.261.262.263.264.265.266.267.268.269.270.271.272.273.274.27

In [18]:
# Calcolo il rapporto e gli anni di partecipazione
for i in swimmers:
    i.fill_partecipation()
    i.calc_ratio(starting_year=Year(2000, 2001), ending_year=Year(2017, 2018))
    print(i)

Name: COLOMBO MARCO Class: 1960 Ratio: 0.35294117647058826
Name: BORDONALI BERNARDO GIACOMO Class: 1961 Ratio: 0.5882352941176471
Name: DI IACOVO LUCA Class: 1967 Ratio: 0.47058823529411764
Name: CARMINATI PAOLO Class: 1964 Ratio: 0.35294117647058826
Name: LAVEGLIA ANTONELLO Class: 1966 Ratio: 0.5294117647058824
Name: SONDALI GIAN LUCA Class: 1966 Ratio: 0.4117647058823529
Name: BORRELLI FABRIZIO Class: 1966 Ratio: 0.47058823529411764
Name: LA SALA GENNARO Class: 1961 Ratio: 0.47058823529411764
Name: MICHELOTTI LEONARDO Class: 1965 Ratio: 0.058823529411764705
Name: TENDERINI MARCO Class: 1967 Ratio: 0.29411764705882354
Name: VIGNERI LUCA Class: 1967 Ratio: 0.11764705882352941
Name: NANETTI DAMIANO Class: 1962 Ratio: 0.29411764705882354
Name: BRUNORI ROBERTO Class: 1964 Ratio: 0.47058823529411764
Name: SALVATORI ALESSANDRO Class: 1965 Ratio: 0.23529411764705882
Name: RODELLA MAURO Class: 1963 Ratio: 0.47058823529411764
Name: LEONE MARCO Class: 1965 Ratio: 0.23529411764705882
Name: GERBI

Name: BATTISTA BRUNO Class: 1960 Ratio: 0.23529411764705882
Name: TIRELLI FULVIO Class: 1965 Ratio: 0.4117647058823529
Name: DE ANGELIS MAURIZIO Class: 1961 Ratio: 0.4117647058823529
Name: PASCARELLA GIAMPAOLO Class: 1962 Ratio: 0.058823529411764705
Name: BUTTI ANDREA Class: 1967 Ratio: 0.058823529411764705
Name: MAURI RENZO Class: 1964 Ratio: 0.058823529411764705
Name: SORDI MARIO Class: 1967 Ratio: 0.11764705882352941
Name: FATTORE ALESSANDRO Class: 1966 Ratio: 0.058823529411764705
Name: STELLA EROS Class: 1932 Ratio: 0.058823529411764705
Name: ANSELMI ENRICO Class: 1966 Ratio: 0.11764705882352941
Name: OMATI MICHELE Class: 1965 Ratio: 0.058823529411764705
Name: SANTOMIERI GIANMATTEO Class: 1960 Ratio: 0.7058823529411765
Name: FILIPPI FRANCESCO Class: 1955 Ratio: 0.29411764705882354
Name: CALLONE LEARDO Class: 1945 Ratio: 0.35294117647058826
Name: BALBO ANDREA Class: 1964 Ratio: 0.11764705882352941
Name: PIVA ALESSANDRO SIMONE Class: 1963 Ratio: 0.35294117647058826
Name: SANTINI GABR

In [19]:
# Salva nel database i nuotatori con pickle
with open("nuotatori_"+Stile+"_db.pkl","wb") as pkl:
    pickle.dump(swimmers, pkl)